In [ ]:
from dolfin import *
import matplotlib.pyplot as plt
import matplotlib as m
import os
from ufl import nabla_div

# Constants
ys = 1
t = 0
dt = 0.00001
T = 1
rho1 = 1
rho2 = 2
mu1 = 1
mu2 = 1
lamda1 = 1
lamda2 = 1

# Mesh, functionspace, functions
mesh = RectangleMesh(Point(0.0, 0.0), Point(2.0, 2.0), 200,200)
V = VectorFunctionSpace(mesh, "Lagrange", 1)
D = VectorFunctionSpace(mesh, "DG", 0)
u = TrialFunction(V)
v = TestFunction(V)

eps = DOLFIN_EPS

# Define Subdomains
#cf = CellFunction("size_t", mesh, 0)
#domain1 = AutoSubDomain(lambda x: x[1] > ys)
#domain1.mark(cf,1)

# Define variable constants
#rhof = Expression(("x[1] > ys ? rho1 : rho2",
#                   "x[1] > ys ? rho1 : rho2"),
#                  ys=ys,rho1=rho1,rho2=rho2)
rhof   = Expression(("x[1] > ys? rho1 : rho2", "0.0"),
                  degree=0, ys=ys,rho1=rho1,rho2=rho2)
muf    = Expression(("x[1] > ys? mu1 : mu2", "0.0"),
                 degree=0, ys=ys,mu1=mu1,mu2=mu2)
lamdaf = Expression(("x[1] > ys? lamda1 : lamda2", "0.0"),
                    degree=0, ys=ys,lamda1=lamda1,lamda2=lamda2)
stepf  = Expression(("x[1] > ys? pow(dt,2)/rho1 : pow(dt,2)/rho2", "0.0"), 
                   degree=0, ys=ys,dt=dt,rho1=rho1,rho2=rho2)

#create the fields...note that we do this dot product trick because
# we have a vector-valued space
# TODO: Change this to a scalar-valued space...D does NOT need to be defined
#       as a vector field since it has nothing to do with the displacement vector

#NOTE: lamda is obviously mispelled, but that's because lambda is a keyword
#        of Python
e_x   = interpolate(Expression(('1.0', '0.0'), degree=1), D)
rho   = dot(e_x, interpolate(rhof, D))
mu    = dot(e_x, interpolate(muf, D))
lamda = dot(e_x, interpolate(lamdaf, D))
step  = dot(e_x, interpolate(stepf, D))

c = plot(rho)
plt.colorbar(c)
plt.show()
input('rho plotted')

# Stress tensor
def sigma(u, lamda, mu):
    return lamda*nabla_div(u)*Identity(2) + mu*(grad(u) + grad(u).T)

# Initial conditions
Ixy = Constant((1.0,0.0))
Vxy = Constant((1.0,0.0))
u2 = interpolate(Ixy, V)
u1 = interpolate(Vxy, V)

# Variational forms
#F = inner(u,v)*dx - 2*inner(u1,v)*dx + inner(u2,v)*dx + step * inner(sigma(u1,lamda,mu), grad(v)) * dx
A = inner(u,v) * dx 


t = 2*dt
#left = assemble(lhs(F))
u = Function(V)

N = len(u.vector()[:])

def plot_comp(fnc, i, W, the_title, the_min, the_max):
    cdict = {
        'red'  :  ( (0.0, 0.25, .25), (0.02, .59, .59), (1., 1., 1.)),
        'green':  ( (0.0, 0.0, 0.0), (0.02, .45, .45), (1., .97, .97)),
        'blue' :  ( (0.0, 1.0, 1.0), (0.02, .75, .75), (1., 0.45, 0.45))
    }
    cm = m.colors.LinearSegmentedColormap('my_colormap', cdict, 1024)
    cm = 'viridis'
    
    f = fnc
    if i == 0:
        e_x = interpolate(Expression(('1.0', '0.0'), degree=1), W)
        f = dot(fnc, e_x)
    elif i == 1:
        e_y = interpolate(Expression(('0.0', '1.0'), degree=1), W)
        f = dot(fnc, e_y)
    c = plot(f, cmap=cm)
    plt.colorbar(c)
    plt.title(the_title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    
def get_comp(fnc, i, W):
    if( i == 0 ):
        return interpolate(dot(fnc, interpolate(Expression(('1.0', '0.0'), degree=1), W)), W)
    else:
        return interpolate(dot(fnc, interpolate(Expression(('0.0', '1.0'), degree=1), W)), W)
    
ofile = File('soln.pvd')
# Time stepping
i = 0
j = 1

def boundary(x):
    return min(x[0], x[1]) < DOLFIN_EPS or max(x[0],x[1]) > 2.0 - DOLFIN_EPS

while t < T + DOLFIN_EPS:
    #plot_comp(u, 0, V, 'X component for t = %s'%(t), -1.0, 1.0)
    #plot_comp(u, 1, V, 'Y component for t = %s'%(t), -1.0, 1.0)
    print('writing at t = %s'%(t))
    #ofile << u
    if( (i % 10) == 0 ):
        plot_comp(u,0,V, 'X component for plot #%s'%(j), -1.0, 1.0)
        j = j+1
    #right = assemble(rhs(F))
    L = 2 * inner(u1, v) * dx + inner(u2, v) * dx + \
        step * inner(sigma(u1, lamda, mu), grad(v)) * dx
    
    print("solving at time step t=%g" % t)
    bc = DirichletBC(V, Constant((0.0,0.0)), boundary)
    solve(A == L, u, bc)
    #solve(left, u.vector(), right)
    t = t + dt
    i = i + 1

    u2.assign(u1)
    u1.assign(u)
    
os.system("sed -i \"\" \'s/UInt32/Int32/g\' *.vtu")
os.system("sed -i \"\" \'s/f_[0-9]*/f/g\' *.vtu")

In [12]:
#below is the code that is from the tutorial handbook for Fenics

from fenics import *
from ufl import nabla_div
# Scaled variables
L = 1
W = 0.2
mu = 1
rho = 1
delta = W/L
gamma = 0.4*delta**2 
beta = 1.25
lambda_ = beta 
g = gamma

# Create mesh and define function space
mesh = BoxMesh(Point(0, 0, 0), Point(L, W, W), 10, 3, 3)
V = VectorFunctionSpace(mesh, "P", 1)

# Define boundary condition
tol = 1E-14

def clamped_boundary(x, on_boundary):
    return on_boundary and x[0] < tol

bc = DirichletBC(V, Constant((0, 0, 0)), clamped_boundary) # Define strain and stress
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T) #return sym(nabla_grad(u))

def sigma(u):
    return lambda_*nabla_div(u)*Identity(d) + 2*mu*epsilon(u)

# Define variational problem
u = TrialFunction(V)
d = u.geometric_dimension() # space dimension 
v = TestFunction(V)
f = Constant((0, 0, -rho*g))
T = Constant((0, 0, 0))
a = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx + dot(T, v)*ds
# Compute solution
u = Function(V)
solve(a == L, u, bc)
# Plot solution
vtkfile = File("soln.pvd") 
vtkfile << u

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.linspace(0,1,1000)
y = np.linspace(0,1,1000)
X,Y = np.meshgrid(x,y)

Z = np.sqrt(X ** 2 + Y ** 2)

plt.plot(X,Y,Z)

In [ ]:
W = VectorFunctionSpace(mesh, "Lagrange", 1)
f = interpolate(Expression(('sin(x[0])', 'exp(-x[1])'), degree=1), W)
e_x = interpolate(Expression(('1.0', '0.0'), degree=1), W)
e_y = interpolate(Expression(('0.0', '1.0'), degree=1), W)

c = plot(dot(f,e_x), mode='color')
plt.colorbar(c)
plt.title("X component of test field")
plt.show()

c = plot(dot(f,e_y))
plt.colorbar(c)
plt.title('Y component of test field')
plt.show()

c = plot(f)
plt.colorbar(c)
plt.title('Full test vector field')

In [ ]:
print("sed -i \"\" \'s/UInt32/Int32/g\' *.vtu")

In [ ]:
print("sed -i \"\" \'s/f_[0-9]*/f/g\' *.vtu")

In [ ]:
max(1,2)